# HW3: Netflix Data Analysis

In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import scipy.sparse
import random

In [3]:
ratings = scipy.sparse.load_npz('sample_matrix.npz')

In [4]:
ratings_test = scipy.sparse.load_npz('sample_matrix_test.npz')

In [5]:
test_value = np.load('test_value.npz')['arr_0']
test_movie = np.load('test_movie.npz')['arr_0']
test_user = np.load('test_user.npz')['arr_0']
train_value = np.load('train_value.npz')['arr_0']
train_movie = np.load('train_movie.npz')['arr_0']
train_user = np.load('train_user.npz')['arr_0']

In [6]:
# Get features
movie_avg = np.zeros(5000)
movie_revs = np.zeros(5000)
# movie_pos = np.zeros(5000)
# movie_neg = np.zeros(5000)
user_avg = np.zeros (10000)
user_revs = np.zeros(10000)
# user_pos = np.zeros(10000)
# user_neg = np.zeros(10000)

for i in range(5000):
    movie_slice = ratings[i, :]
    movie_avg[i] = csr_matrix.mean(movie_slice)
    movie_revs[i] = csr_matrix.count_nonzero(movie_slice)
#     count_neg = 0
#     count_pos = 0
#     for j in range(movie_slice.shape[1]):
#         if movie_slice[0,j] > 4:
#             count_pos+=1
#         elif (movie_slice[0,j] < 2 and movie_slice[0,j] > 0):
#             count_neg+=1
#     movie_pos[i] = count_pos
#     movie_neg[i] = count_neg
    
for i in range(10000):
    user_slice = ratings[:, i]
    user_avg[i] = csr_matrix.mean(user_slice)
    user_revs[i] = csr_matrix.count_nonzero(user_slice)
#     count_neg = 0
#     count_pos = 0
#     for j in range(user_slice.shape[1]):
#         if user_slice[0,j] > 4:
#             count_pos+=1
#         elif (user_slice[0,j] < 2 and user_slice[0,j] > 0):
#             count_neg+=1
#     user_pos[i] = count_pos
#     user_neg[i] = count_neg

In [7]:
#Movie avg, user avg, # movie reviews, # user reviews, # positive movie reviews, # negative movie review
# #positive user reiews, # negative user reviews
movie_avg_train = np.zeros(len(train_value))
user_avg_train = np.zeros(len(train_value))
movie_revs_train = np.zeros(len(train_value))
user_revs_train = np.zeros(len(train_value))
# movie_pos_train = np.zeros(len(train_value))
# user_pos_train = np.zeros(len(train_value))
# movie_neg_train = np.zeros(len(train_value))
# user_neg_train = np.zeros(len(train_value))

for i in range(len(train_value)):
    movie_avg_train[i] = movie_avg[train_movie[i]]
    user_avg_train[i] = user_avg[train_user[i]]
    movie_revs_train[i] = movie_revs[train_movie[i]]
    user_revs_train[i] = user_revs[train_user[i]]
#     movie_pos_train[i] = movie_pos[train_moive[i]]
#     user_pos_train[i] = user_pos[train_user[i]]
#     movie_neg_train[i] = movie_neg[train_movie[i]]
#     user_neg_train[i] = user_neg[train_movie[i]]

# features_train = np.vstack((movie_avg_train, user_avg_train, movie_revs_train, user_revs_train,
#                             movie_pos_train, user_pos_train, movie_neg_train, user_neg_train)).T

features_train = np.vstack((movie_avg_train, user_avg_train, movie_revs_train, user_revs_train)).T


movie_avg_test = np.zeros(len(test_value))
user_avg_test = np.zeros(len(test_value))
movie_revs_test = np.zeros(len(test_value))
user_revs_test = np.zeros(len(test_value))
user_revs_test = np.zeros(len(test_value))
# movie_pos_test = np.zeros(len(test_value))
# user_pos_test = np.zeros(len(test_value))
# movie_neg_test = np.zeros(len(test_value))
# user_neg_test = np.zeros(len(test_value))

for i in range(len(test_value)):
    movie_avg_test[i] = movie_avg[test_movie[i]]
    user_avg_test[i] = user_avg[test_user[i]]
    movie_revs_test[i] = movie_revs[test_movie[i]]
    user_revs_test[i] = user_revs[test_user[i]]
#     movie_pos_test[i] = movie_pos[test_moive[i]]
#     user_pos_test[i] = user_pos[test_user[i]]
#     movie_neg_test[i] = movie_neg[test_movie[i]]
#     user_neg_test[i] = user_neg[test_movie[i]]

# features_test = np.vstack((movie_avg_test, user_avg_test, movie_revs_test, user_revs_test,
#                            movie_pos_test, user_pos_test, movie_neg_test, user_neg_test)).T
features_test = np.vstack((movie_avg_test, user_avg_test, movie_revs_test, user_revs_test)).T

In [8]:
def MSE(predictions, test_set):
    summation = 0
    for i in range(len(predictions)):
        summation += (predictions[i] - test_set[i])**2
    return summation/len(predictions)

In [9]:
def MSE_non_zero(predictions, test_set):
    summation = 0
    non_zero = 0
    for i in range(len(predictions)):
        value = test_set[i]
        if value > 0:
            summation += (predictions[i] - value)**2
            non_zero += 1
    return summation/non_zero

In [ ]:
print MSE(movie_avg_test, test_value)
print MSE(user_avg_test, test_value)
print MSE_non_zero(movie_avg_test, test_value)
print MSE_non_zero(user_avg_test, test_value)

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(features_train, train_value)
predictions = lr.predict(features_test)

In [ ]:
print lr.score(features_test, test_value)

In [ ]:
print MSE(predictions, test_value)
print MSE_non_zero(predictions, test_value)

In [ ]:
from sklearn.linear_model import Ridge

ridge = Ridge()
ridge.fit(features_train, train_value)
predictions = ridge.predict(features_test)

In [ ]:
print ridge.score(features_test, test_value)

In [ ]:
print MSE(predictions, test_value)
print MSE_non_zero(predictions, test_value)

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(features_train, train_value)
predictions = lasso.predict(features_test)

In [ ]:
print MSE(predictions, test_value)
print MSE_non_zero(predictions, test_value)

In [ ]:
print lasso.score(features_test, test_value)

In [10]:
from sklearn.decomposition import TruncatedSVD

In [11]:
# This file consists of titles and release years associated with each ID
movie_titles = pd.read_csv('movie_titles.txt', header = None, names = ['ID','Year','Name'])
print(movie_titles.head())

   ID    Year                          Name
0   1  2003.0               Dinosaur Planet
1   2  2004.0    Isle of Man TT 2004 Review
2   3  1997.0                     Character
3   4  1994.0  Paula Abdul's Get Up & Dance
4   5  2004.0      The Rise and Fall of ECW


In [49]:
n_components = 10
svd = TruncatedSVD(n_components = n_components)

In [50]:
Z = svd.fit_transform(ratings_test)

In [51]:
components = svd.components_

In [52]:
print Z.shape

(5000, 10)


In [53]:
from sklearn.cluster import KMeans
clusters = 20
kmeans = KMeans(init='k-means++', n_init=10, n_clusters=clusters)
kmeans.fit(Z)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=20, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [54]:
kmeans.labels_.shape

(5000,)

In [55]:
labels = kmeans.labels_
cluster_map = []
for i in range(clusters):
    cluster_map.append([])
for i in range(5000):
    cluster_map[labels[i]].append(i)
for i in range(len(cluster_map)):
    print len(cluster_map[i])

3890
19
42
122
5
17
18
57
27
8
54
3
562
3
4
32
116
3
13
5


In [56]:
user_cluster_map={}    

similar_users_avg_train = np.zeros(len(train_value))

for i in range(len(train_value)):
    if i % 1000 == 0:
        print str(i) + 'one'
    user = train_user[i]
    label = labels[train_movie[i]]
    map_key = str(user) + ':' + str(label)
    if i % 1000 == 0:
        print str(i) + 'two'
    if map_key in user_cluster_map:
        if i % 1000 == 0:
            print str(i) + "if"
        similar_users_avg_train[i] = user_cluster_map[map_key]
    else:
        if i % 1000 == 0:
            print i
        user_sum = 0
        similar_movies = cluster_map[label]
        if i % 1000 == 0:
            print len(similar_movies)
        for j in range(len(similar_movies)):
            user_sum += ratings_test[train_movie[similar_movies[j]],user]
        similar_users_avg_train[i] = user_sum / len(similar_movies)
        user_cluster_map[map_key] = similar_users_avg_train[i]

similar_users_avg_test = np.zeros(len(test_value))

for i in range(len(test_value)):
    user = test_user[i]
    label = labels[test_movie[i]]
    map_key = str(user) + ':' + str(label)
    if map_key in user_cluster_map:
        similar_users_avg_test[i] = user_cluster_map[map_key]
    else:
        user_sum = 0
        similar_movies = cluster_map[label]
        for j in range(len(similar_movies)):
            user_sum += ratings_test[test_movie[similar_movies[j]],user]
        similar_users_avg_test[i] = user_sum / len(similar_movies)
        user_cluster_map[map_key] = similar_users_avg_test[i]
    
    #average that the user rated similar movies
    

0one
0two
0
32
1000one
1000two
1000
32
2000one
2000two
2000
32
3000one
3000two
3000
32
4000one
4000two
4000
32
5000one
5000two
5000
32
6000one
6000two
6000
32
7000one
7000two
7000
32
8000one
8000two
8000
3890


KeyboardInterrupt: 

In [ ]:
features_train_complex = similar_users_avg_train.T
features_test_complex = similar_users_avg_test.T

In [ ]:
lr = LinearRegression()
lr.fit(features_train_complex, train_value)
predictions = lr.predict(features_test)

In [ ]:
print MSE(predictions, test_value)
print MSE_non_zero(predictions, test_value)